In [6]:
# imports
import keras
import pandas as pd
import torch_cnn as tcnn
import keras_cnn as kcnn
import data_processing as d  
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import train_test_split   

In [ ]:
# Data retieval
X, y = d.get_encoded_data()

# Keras CNN Data
X_train, X_test, y_train, y_test = d.splitData(X, y)

# Convert data to Tensors to Incease Computation time for Torch CNN
X_tensor_train,Y_tensor_train,X_tensor_test,Y_tensor_test  = d.toTensors(X_train, y_train, X_test, y_test)

# Change Y data for SVM
svm_X_train, svm_X_test, svm_y_train, svm_y_test =  train_test_split(X, d.get_SVM_y(), test_size=0.33, random_state=1)

In [ ]:
# Variables

EPOCHS = 1000 # arbitrary

# Keras CNN
LAYERS = 52 # arbitrary number for now

# Torch CNN
N_FEATURES = X_train.shape[1] # Number of features for the input layer
NUM_ROWS_TRAINING = X_train.shape[0] # Number of rows
N_HIDDEN = N_FEATURES * 10 # Size of first linear layer
N_CNN_KERNEL, MAX_POOL_KERNEL = 3, 4 # CNN kernel size

In [ ]:
# Keras CNN
model = kcnn.getModel()
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer, 'mse', metrics=[keras.metrics.Accuracy(), 
                                                keras.metrics.MeanSquaredError(),
                                                keras.metrics.Precision(), 
                                                keras.metrics.Recall(),
                                                keras.metrics.RootMeanSquaredError()])
    
history = model.fit(X_train,y_train, batch_size=64,epochs=EPOCHS)
    
for metric in history:
    kcnn.plot(history, EPOCHS, metric, "Training")
    
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)
    
for metric in results:
    kcnn.plot(results, EPOCHS, metric, "Testing")

In [ ]:
# Torch CNN

# Build CNN
net = tcnn.CNN(n_feature=N_FEATURES, n_hidden=N_HIDDEN, n_output=13, n_cnn_kernel=N_CNN_KERNEL)   # define the network    
# Train and Test
all_losses,pred_y,target_y = tcnn.trainTestCNN(net,X_tensor_train,Y_tensor_train,X_tensor_test,Y_tensor_test)

In [ ]:
# SVM
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

clf.fit(X_train,  y_train)
for x in range(0,EPOCHS):
    predictions=clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(accuracy)

In [27]:
df = pd.read_csv('datasets/VirusSample.csv')
df = df.drop(columns=['file'])
pd.set_option('max_colwidth', 150)
shuffle(df).head(20)

,api,class
9129,"LoadLibraryA,GetProcAddress,ExitProcess,RegCloseKey,VariantClear,CharNextA",Worms
5152,"LoadLibraryA,GetProcAddress,VirtualProtect,VirtualAlloc,VirtualFree,ExitProcess,atoi,PathFileExistsA,wsprintfA",Trojan
5096,"GetProcAddress,GetModuleHandleA,LoadLibraryA,GetKeyboardType,RegQueryValueExA,SysFreeString,RegSetValueExA,VerQueryValueA,UnrealizeObject,CreateWi...",Trojan
8686,"ExitProcess,GetEnvironmentStringsA,CloseHandle,GetSystemDirectoryA,OpenMutexA,RtlUnwind,WinExec,_fdopen,_open_osfhandle,fclose,_cexit,malloc,print...",Backdoor
4628,"DeleteCriticalSection,EnterCriticalSection,ExitProcess,FreeLibrary,GetLastError,GetModuleHandleA,GetProcAddress,GlobalAlloc,GlobalFree,InitializeC...",Trojan
445,"LoadLibraryA,GetProcAddress,ExitProcess,GetAce,LZSeek,CoCreateGuid,SetRect,VerQueryValueA,WSAGetLastError",Adware
5463,"MethCallEngine,EVENT_SINK_AddRef,DllFunctionCall,EVENT_SINK_Release,EVENT_SINK_QueryInterface,__vbaExceptHandler,ProcCallEngine",Trojan
22,"ObjectOpenAuditAlarmA,GetCurrentProcess,GetCapture",Virus
3055,"ObjectDeleteAuditAlarmW,VarR8FromDate,GetKBCodePage,GetFocus,ModifyMenuW,NtReplyWaitReceivePort,GetSystemDefaultLCID",Virus
6193,"LoadLibraryA,GetProcAddress,VirtualProtect,VirtualAlloc,VirtualFree,ExitProcess,atoi,PathFileExistsA,wsprintfA",Trojan
